# W_GL_ACCOUNT_D ETL Process
### Staging to Dimension - SCD Type 1 with Change Detection
**ODI Package:** SILOS_SIL_GLACCOUNTDIMENSION  
**IKM:** BIAPPS Oracle Slowly Changing Dimension (Type 1)

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Create Flow Table I$_2625278_1 with change detection and user lookups
DROP VIEW IF EXISTS I$_2625278_1;

CREATE OR REPLACE TEMP VIEW I$_2625278_1 AS
SELECT
  C.*,
  CASE
    WHEN T.ROW_WID IS NOT NULL
      AND (T.CHANGED_ON_DT = C.CHANGED_ON_DT OR (T.CHANGED_ON_DT IS NULL AND C.CHANGED_ON_DT IS NULL))
    THEN 'N'  -- No change
    WHEN T.ROW_WID IS NOT NULL THEN 'U'  -- Update
    ELSE 'I'  -- Insert
  END AS IND_UPDATE
FROM (
  SELECT
    COALESCE(LKP_USER_CREATED.ROW_WID, 0) AS CREATED_BY_WID,
    COALESCE(LKP_USER_CHANGED.ROW_WID, 0) AS CHANGED_BY_WID,
    DS.CHART_OF_ACCOUNTS,
    DS.CONCATENATED_SEGMENT_VALUE,
    DS.BALANCING_SEGMENT_NUM,
    DS.BALANCING_SEGMENT_ATTRIB,
    DS.COST_CENTER_NUM,
    DS.COST_CENTER_ATTRIB,
    DS.NATURAL_ACCOUNT_NUM,
    DS.NATURAL_ACCOUNT_ATTRIB,
    DS.RECON_TYPE_CODE,
    DS.ACTIVE_FLG,
    DS.CREATED_ON_DT,
    DS.CHANGED_ON_DT,
    DS.SRC_EFF_FROM_DT,
    DS.DELETE_FLG,
    DS.DATASOURCE_NUM_ID,
    DS.INTEGRATION_ID,
    DS.TENANT_ID,
    DS.X_CUSTOM
  FROM workspace.oracle_edw.W_GL_ACCOUNT_DS DS
  LEFT OUTER JOIN workspace.oracle_edw.W_USER_D LKP_USER_CREATED
    ON DS.CREATED_BY_ID = LKP_USER_CREATED.INTEGRATION_ID
    AND DS.DATASOURCE_NUM_ID = LKP_USER_CREATED.DATASOURCE_NUM_ID
    AND DS.CREATED_ON_DT >= LKP_USER_CREATED.EFFECTIVE_FROM_DT
    AND DS.CREATED_ON_DT < LKP_USER_CREATED.EFFECTIVE_TO_DT
  LEFT OUTER JOIN workspace.oracle_edw.W_USER_D LKP_USER_CHANGED
    ON DS.CHANGED_BY_ID = LKP_USER_CHANGED.INTEGRATION_ID
    AND DS.DATASOURCE_NUM_ID = LKP_USER_CHANGED.DATASOURCE_NUM_ID
    AND DS.CHANGED_ON_DT >= LKP_USER_CHANGED.EFFECTIVE_FROM_DT
    AND DS.CHANGED_ON_DT < LKP_USER_CHANGED.EFFECTIVE_TO_DT
) C
LEFT OUTER JOIN workspace.oracle_edw.W_GL_ACCOUNT_D T
  ON C.DATASOURCE_NUM_ID = T.DATASOURCE_NUM_ID
  AND C.INTEGRATION_ID = T.INTEGRATION_ID

In [ ]:
%sql
-- Step 3: Update existing rows (SCD Type 1 - overwrite)
MERGE INTO workspace.oracle_edw.W_GL_ACCOUNT_D AS T
USING (
  SELECT * FROM I$_2625278_1 WHERE IND_UPDATE = 'U'
) S
ON T.DATASOURCE_NUM_ID = S.DATASOURCE_NUM_ID
  AND T.INTEGRATION_ID = S.INTEGRATION_ID
WHEN MATCHED THEN
  UPDATE SET
    T.CREATED_BY_WID = S.CREATED_BY_WID,
    T.CHANGED_BY_WID = S.CHANGED_BY_WID,
    T.CHART_OF_ACCOUNTS = S.CHART_OF_ACCOUNTS,
    T.CONCATENATED_SEGMENT_VALUE = S.CONCATENATED_SEGMENT_VALUE,
    T.BALANCING_SEGMENT_NUM = S.BALANCING_SEGMENT_NUM,
    T.BALANCING_SEGMENT_ATTRIB = S.BALANCING_SEGMENT_ATTRIB,
    T.COST_CENTER_NUM = S.COST_CENTER_NUM,
    T.COST_CENTER_ATTRIB = S.COST_CENTER_ATTRIB,
    T.NATURAL_ACCOUNT_NUM = S.NATURAL_ACCOUNT_NUM,
    T.NATURAL_ACCOUNT_ATTRIB = S.NATURAL_ACCOUNT_ATTRIB,
    T.RECON_TYPE_CODE = S.RECON_TYPE_CODE,
    T.ACTIVE_FLG = S.ACTIVE_FLG,
    T.CREATED_ON_DT = CAST(S.CREATED_ON_DT AS TIMESTAMP),
    T.CHANGED_ON_DT = CAST(S.CHANGED_ON_DT AS TIMESTAMP),
    T.SRC_EFF_FROM_DT = S.SRC_EFF_FROM_DT,
    T.DELETE_FLG = S.DELETE_FLG,
    T.TENANT_ID = S.TENANT_ID,
    T.X_CUSTOM = S.X_CUSTOM,
    T.ETL_PROC_WID = 2625278

In [ ]:
%sql
-- Step 4: Insert new rows
INSERT INTO workspace.oracle_edw.W_GL_ACCOUNT_D
(
  ROW_WID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  RECON_TYPE_CODE,
  ACTIVE_FLG,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  ETL_PROC_WID
)
SELECT
  row_number() OVER (ORDER BY INTEGRATION_ID) + 
  COALESCE((SELECT MAX(ROW_WID) FROM workspace.oracle_edw.W_GL_ACCOUNT_D), 0) AS ROW_WID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  RECON_TYPE_CODE,
  ACTIVE_FLG,
  CAST(CREATED_ON_DT AS TIMESTAMP) AS CREATED_ON_DT,
  CAST(CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  2625278 AS ETL_PROC_WID
FROM I$_2625278_1
WHERE IND_UPDATE = 'I'

In [ ]:
%sql
-- Step 5: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SILOS_SIL_GLACCOUNTDIMENSION' AS PACKAGE_NAME,
    'W_GL_ACCOUNT_D' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2625278 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 6: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2625278 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 7: Drop work table (temp view)
DROP VIEW IF EXISTS I$_2625278_1;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL